In [ ]:
%pip install tensorflow-serving-api keras-image-helper

In [2]:
import tensorflow as tf

2024-12-30 00:13:37.938499: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-30 00:13:38.031785: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 00:13:38.031833: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 00:13:38.031859: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 00:13:38.048779: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-30 00:13:38.050569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
import grpc

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [4]:
host = 'localhost:8500'

#GRPC_MAX_RECEIVE_MESSAGE_LENGTH = 4096 * 4096 * 3

In [5]:
channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [6]:
from keras_image_helper import create_preprocessor

In [7]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [8]:
url = "https://bit.ly/al-gaib"

In [9]:
X = preprocessor.from_url(url)

In [13]:
grpc_request = predict_pb2.PredictRequest()

grpc_request.model_spec.name = 'clothing-model'
grpc_request.model_spec.signature_name = 'serving_default'

grpc_request.inputs['input_2'].CopyFrom(tf.make_tensor_proto(X, shape=X.shape))

In [14]:
grpc_response = stub.Predict(grpc_request, timeout=20.0)
preds = grpc_response.outputs['dense_1'].float_val

In [15]:
classes = [
        'dress',
        'hat',
        'longsleeve',
        'outwear',
        'pants',
        'shirt',
        'shoes',
        'shorts',
        'skirt',
        't-shirt'
        ]

In [16]:
dict(zip(classes, preds))

{'dress': -3.046570062637329,
 'hat': -3.63806414604187,
 'longsleeve': -2.2050061225891113,
 'outwear': -1.6138986349105835,
 'pants': 3.2289621829986572,
 'shirt': -1.6451092958450317,
 'shoes': -2.3591649532318115,
 'shorts': 1.1146421432495117,
 'skirt': -0.5203550457954407,
 't-shirt': -2.2242445945739746}

* Docker command to run the tensorflow serving
```
docker run -it --rm  -p 8500:8500 -p 8501:8501 -v "$(pwd)/clothing-model:/models/clothing-model/1" -e MODEL_NAME="clothing-model"  tensorflow/serving:2.14.0